In [0]:
import pyspark.sql.functions as sf

In [0]:
df_shop = spark.read.table("runescape.02_silver.shop_prices_enriched")

In [0]:
df_shop.display()

In [0]:
# Read data from 'runescape.01_bronze.1h_prices'
df_hourly = spark.read.table("runescape.01_bronze.1h_prices")
df_hourly.count()

In [0]:
# hourly data for trading sticks
df_hourly_ts = df_hourly.filter("id == 6306")
df_hourly_ts = df_hourly_ts.select("time", "avg1HourHigh", "avg1HourLow")
df_hourly_ts = df_hourly_ts.withColumnsRenamed({"avg1HourHigh": "avg1HourHigh_ts", "avg1HourLow": "avg1HourLow_ts"})

In [0]:
df = df_shop.join(df_hourly, "id")

In [0]:
# gabooty store (gabooty store currency is trading sticks not gp)
df_gabooty = df.filter("store == 'gabooty'")
df_gabooty = df_gabooty.withColumnsRenamed({"buy_price": "buy_price_ts", "sell_price": "sell_price_ts"})

In [0]:
# gabooty store currency is trading sticks not gp
df = df.filter("store != 'gabooty'")


In [0]:
df = df.withColumn("profit_sell_to_store_high", sf.col("buy_price") - sf.col("avg1HourHigh"))
df = df.withColumn("profit_sell_to_store_low", sf.col("buy_price") - sf.col("avg1Hourlow"))
df = df.withColumn("profit_buy_from_store_high", sf.col("avg1HourHigh") - sf.col("sell_price"))
df = df.withColumn("profit_buy_from_store_low", sf.col("avg1Hourlow") - sf.col("sell_price"))

In [0]:
df.display()

In [0]:
# profit for selling Amulet of Strength to Davon over time
df_aos = df.filter("id == 1725")
df_aos.display()

Databricks visualization. Run in Databricks to view.

In [0]:
# profit for selling to Davon
df_davon = df.filter("store == 'davon'")
df_davon = df_davon.filter("id <> 1718") # holy symbols are never profitable
df_davon.display()

Databricks visualization. Run in Databricks to view.

In [0]:
# profit for buying from baba
df_baba = df.filter("store == 'baba'")
# Mind rune, Air rune, Water rune, Fire rune, Earth rune are never profitable
df_baba = df_baba.filter("id NOT IN ('558','556','555','554','557')")
df_baba.display()

Databricks visualization. Run in Databricks to view.

In [0]:
# profit for selling to baba
df_baba = df.filter("store == 'baba'")
# Mind rune, Air rune, Water rune, Fire rune, Earth rune are never profitable
df_baba = df_baba.filter("id NOT IN ('558','556','555','554','557')")
# Not profitable
df_baba = df_baba.filter("id NOT IN ('1387','1391','1383','560','562','566','9075','565','1381','559','561','563')")
df_baba.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_gabooty = df_gabooty.join(df_hourly_ts, "time")


In [0]:


df_gabooty = df_gabooty.withColumn("profit_sell_to_store_high_ts_high", sf.col("buy_price_ts") * sf.col("avg1HourHigh_ts") - sf.col("avg1HourHigh"))
df_gabooty = df_gabooty.withColumn("profit_sell_to_store_high_ts_low", sf.col("buy_price_ts") * sf.col("avg1HourLow_ts") - sf.col("avg1HourHigh"))
df_gabooty = df_gabooty.withColumn("profit_sell_to_store_low_ts_high", sf.col("buy_price_ts") * sf.col("avg1HourHigh_ts") - sf.col("avg1HourLow"))
df_gabooty = df_gabooty.withColumn("profit_sell_to_store_low_ts_low", sf.col("buy_price_ts") * sf.col("avg1HourLow_ts") - sf.col("avg1HourLow"))



In [0]:
df_gabooty_sell_to_store = df_gabooty
df_gabooty_sell_to_store = df_gabooty_sell_to_store.filter("id NOT IN ('6351','6349','6357','6311','6317','6369','6313','6347','6367','6377','6353','6345','6341','6373','6375')")

Databricks visualization. Run in Databricks to view.

In [0]:
%skip

df_gabooty = df_gabooty.withColumn("profit_buy_from_store_high", sf.col("avg1HourHigh") - sf.col("sell_price_ts"))
df_gabooty = df_gabooty.withColumn("profit_buy_from_store_low", sf.col("avg1Hourlow") - sf.col("sell_price_ts"))